### 索引持久化

In [27]:
from ollama import Client
import numpy as np
import faiss

client = Client(host='http://localhost:11434')
 # 调用嵌入模型生成词向量
def get_embedding(text, model="dengcao/Qwen3-Embedding-8B:Q5_K_M"):
    response = client.embeddings(model=model, prompt=text)
    return np.array(response['embedding'], dtype=np.float32)

# 创建示例向量数据
texts = ["神经网络原理", "深度学习框架", "大模型训练"]
vectors = np.array([get_embedding(t) for t in texts], dtype='float32')

# 创建一个FAISS索引（这里使用最简单的IndexFlatL2索引）
dimension = vectors[0].shape[0]
index = faiss.IndexFlatL2(dimension)

# 添加向量到索引
index.add(vectors)

# 将索引保存到文件
faiss.write_index(index, "my_faiss_index.faiss")

print("FAISS索引已保存到 my_faiss_index.faiss")

FAISS索引已保存到 my_faiss_index.faiss


### 文本映射数据持久化

In [28]:
import pickle

texts = ["神经网络原理", "深度学习框架", "大模型训练"]

# 将文本列表保存到文件
with open("text_mapping.pkl", "wb") as f:
    pickle.dump(texts, f)

print("文本映射已保存到 text_mapping.pkl")

文本映射已保存到 text_mapping.pkl


### 读取FAISS索引

In [29]:
import faiss

loaded_index = faiss.read_index("my_faiss_index.faiss")
print(f"索引加载成功，当前向量数量：{loaded_index.ntotal}")

索引加载成功，当前向量数量：3


### 读取文本映射数据

In [30]:
import pickle

with open("text_mapping.pkl", "rb") as f:
    loaded_texts = pickle.load(f)

print(f"加载的文本列表：{loaded_texts}")

加载的文本列表：['神经网络原理', '深度学习框架', '大模型训练']


### 查询知识库内容

In [31]:
import numpy as np


query = "大语言模型"
# query_vec是查询文本的向量（float32 numpy数组，维度与索引一致）
query_vec = get_embedding(query)

k = 1  # 取最相似的1个结果
D, I = loaded_index.search(np.array([query_vec]), k)

print("距离：", D)
print("索引位置：", I)

for idx in I[0]:
    print(f"匹配文本：{loaded_texts[idx]}")

距离： [[11984.881]]
索引位置： [[2]]
匹配文本：大模型训练
